# Loading the cases that are not in per unit and generating base load 

In [1]:
import numpy as np
import pandas as pd
import scipy.io
from pypower.api import ext2int, int2ext, makeYbus

PATH = "/Users/babaktaheri/Desktop/OLDF/Single-phase"

TEST_CASE = '906'

def load_mat():
    mat = scipy.io.loadmat(f"{PATH}/data/case{TEST_CASE}.mat")

    bus = mat['ans']['bus'][0][0]
    baseMVA = mat['ans']['baseMVA'][0][0][0][0]
    gen = mat['ans']['gen'][0][0]
    branch = mat['ans']['branch'][0][0]
    keys = ['baseMVA', 'bus', 'gen', 'branch']
    values = [baseMVA, bus, gen, branch]
    ppc = dict(zip(keys, values))
    ppc = ext2int(ppc)
    baseMVA, bus, gen, branch = ppc['baseMVA'], ppc['bus'], ppc['gen'], ppc['branch']
    Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)

    return baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc

#def convert_branch(baseMVA, bus, branch):
#    Vbase = bus[0, 7] * 1e3  # Vbase = mpc.bus(1, BASE_KV) * 1e3; in Volts
#    Sbase = baseMVA * 1e6  # Sbase = mpc.baseMVA * 1e6; in VA
#    branch[:, [2, 3]] = branch[:, [2, 3]] / (Vbase**2 / Sbase)  # convert branch impedance

def convert_loads(bus):
    bus[:, [2, 3]] = bus[:, [2, 3]] / 1e3  # convert loads from kW to MW

def calculate_just_loads(baseMVA, bus):
    Just_load = bus[np.logical_or(bus[:, 2] != 0, bus[:, 3] != 0)]
    load_buses = Just_load[:, 0]
    load_buses = np.reshape(load_buses, (len(load_buses), 1))

    P_Load = Just_load[:, 2] / baseMVA
    Q_Load = Just_load[:, 3] / baseMVA
    P_load = np.reshape(P_Load, (len(load_buses), 1))
    Q_load = np.reshape(Q_Load, (len(load_buses), 1))

    return P_load, Q_load

baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc = load_mat()
#convert_branch(baseMVA, bus, branch)
"""Convert loads to MW if they are in kW"""
#convert_loads(bus)
P_load, Q_load = calculate_just_loads(baseMVA, bus)


# Loading the 123 case which is already in per unit

In [15]:
import numpy as np
import pandas as pd
import scipy.io
from pypower.api import ext2int, int2ext, makeYbus
import numpy as np
import matplotlib.pyplot as plt
PATH = "/Users/babaktaheri/Desktop/OLDF/Single-phase"

TEST_CASE           =   'IEEE123'


def load_mat():
    mat = scipy.io.loadmat(f"{PATH}/data/case{TEST_CASE}.mat")

    bus = mat['ans']['bus'][0][0]
    baseMVA = mat['ans']['baseMVA'][0][0][0][0]
    gen = mat['ans']['gen'][0][0]
    branch = mat['ans']['branch'][0][0]
    keys = ['baseMVA', 'bus', 'gen', 'branch']
    values = [baseMVA, bus, gen, branch]
    ppc = dict(zip(keys, values))
    ppc = ext2int(ppc)
    baseMVA, bus, gen, branch = ppc['baseMVA'], ppc['bus'], ppc['gen'], ppc['branch']
    Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)

    return baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc

baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc = load_mat()


def calculate_just_loads (baseMVA, bus):
    Just_load = bus[np.logical_or(bus[:, 2] != 0, bus[:, 3] != 0)]
    load_buses=Just_load[:,0]
    load_buses=np.reshape(load_buses,(len(load_buses),1))
  
    P_Load  = Just_load [:,2]  
    Q_Load  = Just_load [:,3]
    P_load  = np.reshape(P_Load,(len(load_buses),1))    
    Q_load  = np.reshape(Q_Load,(len(load_buses),1)) 

    return P_load, Q_load

# Constant PF scenario generation


In [ ]:
import numpy as np
baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc = load_mat()

"""turn this on for 33 bus but not for cases where it is already in MW"""
#convert_loads(bus)

P_load, Q_load = calculate_just_loads (baseMVA, bus)
print(P_load)
N=1000
def generate_scenarios(P_load, Q_load, N):
    power_factor = P_load / np.sqrt(P_load**2 + Q_load**2)

    P_load_scenarios = np.zeros((N, P_load.shape[0]))
    Q_load_scenarios = np.zeros((N, Q_load.shape[0]))

    for idx in range(P_load.shape[0]):
        if P_load[idx, 0] != 0:
            P_load_scenarios[:, idx] = np.random.normal(P_load[idx, 0], abs(0.35 * P_load[idx, 0]), N)
            Q_load_scenarios[:, idx] = P_load_scenarios[:, idx] * np.tan(np.arccos(power_factor[idx, 0]))

    return P_load_scenarios, Q_load_scenarios

def generate_scenarios_neg(P_load, Q_load, N):
    power_factor = P_load / np.sqrt(P_load**2 + Q_load**2)

    P_load_scenarios = np.zeros((N, P_load.shape[0]))
    Q_load_scenarios = np.zeros((N, Q_load.shape[0]))

    for idx in range(P_load.shape[0]):
        if P_load[idx, 0] != 0:
            P_load_scenarios[:, idx] = np.random.normal(-P_load[idx, 0], abs(0.35 * P_load[idx, 0]), N)
            Q_load_scenarios[:, idx] = P_load_scenarios[:, idx] * np.tan(np.arccos(power_factor[idx, 0]))

    return P_load_scenarios, Q_load_scenarios


P_load_scenarios_pos, Q_load_scenarios_pos = generate_scenarios(P_load, Q_load, N)
P_load_scenarios_neg, Q_load_scenarios_neg = generate_scenarios_neg(P_load, Q_load, N)

P_load_scenarios = np.concatenate ((P_load_scenarios_pos,P_load_scenarios_neg))
Q_load_scenarios = np.concatenate ((Q_load_scenarios_pos,Q_load_scenarios_neg))


np.save(f"{PATH}/data/P_load_all_scenarios_{TEST_CASE}bus.npy", P_load_scenarios)
np.save(f"{PATH}/data/Q_load_all_scenarios_{TEST_CASE}bus.npy", Q_load_scenarios)

# For 141 bus case which is differnt and only has MVA data

In [29]:
import numpy as np
import pandas as pd
import scipy.io
from pypower.api import ext2int, int2ext, makeYbus

PATH = "/Users/babaktaheri/Desktop/OLDF/Single-phase"

TEST_CASE = '141'

def load_mat():
    mat = scipy.io.loadmat(f"{PATH}/data/case{TEST_CASE}.mat")

    bus = mat['ans']['bus'][0][0]
    baseMVA = mat['ans']['baseMVA'][0][0][0][0]
    gen = mat['ans']['gen'][0][0]
    branch = mat['ans']['branch'][0][0]
    keys = ['baseMVA', 'bus', 'gen', 'branch']
    values = [baseMVA, bus, gen, branch]
    ppc = dict(zip(keys, values))
    ppc = ext2int(ppc)
    baseMVA, bus, gen, branch = ppc['baseMVA'], ppc['bus'], ppc['gen'], ppc['branch']
    Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)
    
    """Only For 141 bus case which is differnt and only has MVA data"""
    pf = 0.85
    bus[:, 3] = bus[:, 2] * np.sin(np.arccos(pf))  # MVA to MVAr
    bus[:, 2] = bus[:, 2] * pf  # MVA to MW
    

    return baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc



def convert_loads(bus):
    bus[:, [2, 3]] = bus[:, [2, 3]] / 1e3  # convert loads from kW to MW

def calculate_just_loads(baseMVA, bus):
    Just_load = bus[np.logical_or(bus[:, 2] != 0, bus[:, 3] != 0)]
    load_buses = Just_load[:, 0]
    load_buses = np.reshape(load_buses, (len(load_buses), 1))

    P_Load = Just_load[:, 2] / baseMVA
    Q_Load = Just_load[:, 3] / baseMVA
    P_load = np.reshape(P_Load, (len(load_buses), 1))
    Q_load = np.reshape(Q_Load, (len(load_buses), 1))

    return P_load, Q_load

# Example usage:
baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc = load_mat()
#convert_branch(baseMVA, bus, branch)
convert_loads(bus)
P_load, Q_load = calculate_just_loads(baseMVA, bus)

# High Load Scenario Genration

In [6]:
import numpy as np

N = 30  # Number of scenarios for high loads
k_values_positive = np.arange(1, 2.00001, 1/14)
k_values_negative = -np.flip(k_values_positive)  # Generate negative values by flipping the positive ones

k_values = np.concatenate(( k_values_negative, k_values_positive))

def generate_high_load_scenarios(P_load, Q_load, N, k_values):
    # Initialize the scenarios matrix
    P_load_scenarios = np.zeros((N, P_load.shape[0]))
    Q_load_scenarios = np.zeros((N, Q_load.shape[0]))

    # For every non-zero element in P_load, generate N scenarios for high loads with specified k values.
    for idx in range(P_load.shape[0]):
        if P_load[idx, 0] != 0:
            for i, k in enumerate(k_values[:N]):
                P_load_scenarios[i, idx] = k * P_load[idx, 0]
                Q_load_scenarios[i, idx] = k * Q_load[idx, 0]

    return P_load_scenarios, Q_load_scenarios


P_load_high_scenarios, Q_load_high_scenarios = generate_high_load_scenarios(P_load, Q_load, N, k_values)

np.save(f"{PATH}/data/High_P_load_scenarios_{TEST_CASE}bus.npy", P_load_high_scenarios)
np.save(f"{PATH}/data/High_Q_load_scenarios_{TEST_CASE}bus.npy", Q_load_high_scenarios)

# Compute Injection scenarios

In [4]:
import numpy as np
import pandas as pd
import array
import csv
import math
import os.path
import random
import time
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import scipy.io
from pypower.api import ext2int, int2ext, makeYbus
PATH = "/Users/babaktaheri/Desktop/OLDF/Single-phase"


TEST_CASE           =   '906'


def load_mat():
    mat = scipy.io.loadmat(f"{PATH}/data/case{TEST_CASE}.mat")
    bus = mat['ans']['bus'][0][0]
    baseMVA = mat['ans']['baseMVA'][0][0][0][0]
    gen = mat['ans']['gen'][0][0]
    branch = mat['ans']['branch'][0][0]
    keys = ['baseMVA', 'bus', 'gen', 'branch']
    values = [baseMVA, bus, gen, branch]
    ppc = dict(zip(keys, values))
    ppc = ext2int(ppc)
    baseMVA, bus, gen, branch = ppc['baseMVA'], ppc['bus'], ppc['gen'], ppc['branch']
    Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)

    return baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc


def load_gen_data(filename):
    gen_data = pd.read_csv(filename, sep=',')
    return gen_data


def process_bus_and_gen(bus, gen, baseMVA, branch, ppc):
    Just_load = bus[np.logical_or(bus[:, 2] != 0, bus[:, 3] != 0)]
    load_buses=Just_load[:,0]
    load_buses=np.reshape(load_buses,(len(load_buses),1))

    n_buses = len(ppc['bus'])
    n_gens = len(ppc['gen'])
    n_branches = len(ppc['branch'])
    r = branch[:, 2]
    x = branch[:, 3]
    gen_buses = gen[:, 0]
    gen_values = gen[:, 1]
    Q_values = gen[:, 2]


    Pgen = np.zeros(n_buses)
    Qgen = np.zeros(n_buses)

    for i in range(len(gen_buses)):
        bus_idx = int(gen_buses[i])  
        Pgen[bus_idx] += gen_values[i]
        Qgen[bus_idx] += Q_values[i]

    #Depends on ...    
    Pgen  = Pgen / baseMVA
    Qgen  = Qgen / baseMVA

    Pgen = np.reshape(Pgen,(n_buses,1))    
    Qgen = np.reshape(Qgen,(n_buses,1))    

    ref_bus = np.where(bus[:, 1] == 3)[0][0]

    return load_buses, n_buses, n_gens, n_branches, r, x, Pgen,Qgen, ref_bus


baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc = load_mat()
load_buses, n_buses, n_gens, n_branches, r, x, Pgen,Qgen, ref_bus = process_bus_and_gen(bus, gen, baseMVA, branch, ppc)

def compute_and_save_pinj():
    
    baseMVA, bus, gen, branch, _, _, _, ppc = load_mat()
    P_load_all_scenarios = np.load(f"{PATH}/data/P_load_all_scenarios_{TEST_CASE}bus.npy", allow_pickle=True)
    Q_load_all_scenarios = np.load(f"{PATH}/data/Q_load_all_scenarios_{TEST_CASE}bus.npy", allow_pickle=True)

    load_buses, n_buses, _, _, _, _, Pgen,Qgen, ref_bus = process_bus_and_gen(bus, gen, baseMVA, branch, ppc)

    P_inj_all_scenarios = []
    for s in range(P_load_all_scenarios.shape[0]): 
        aaa = P_load_all_scenarios[s, :]

        Demand = np.zeros(n_buses)
        for i in range(len(load_buses)):
            bus_idx = int(load_buses[i][0])  

            Demand[bus_idx] = aaa[i]
        Demand = np.reshape(Demand, (n_buses, 1))
        P_inj = Pgen - Demand
        P_inj = np.delete(P_inj, ref_bus, 0)
        P_inj = np.asmatrix(P_inj)
        P_inj_all_scenarios.append(P_inj)
        
    Q_inj_all_scenarios = []
    for s in range(Q_load_all_scenarios.shape[0]): 
        aaa = Q_load_all_scenarios[s, :]

        Demand = np.zeros(n_buses)
        for i in range(len(load_buses)):
            bus_idx = int(load_buses[i][0])  

            Demand[bus_idx] = aaa[i]
        Demand = np.reshape(Demand, (n_buses, 1))
        Q_inj = Qgen - Demand
        Q_inj = np.delete(Q_inj, ref_bus, 0)
        Q_inj = np.asmatrix(Q_inj)
        Q_inj_all_scenarios.append(Q_inj)        
        
    np.save(f"{PATH}/data/P_inj_all_scenarios_{TEST_CASE}bus.npy", P_inj_all_scenarios)
    np.save(f"{PATH}/data/Q_inj_all_scenarios_{TEST_CASE}bus.npy", Q_inj_all_scenarios)

compute_and_save_pinj()

# High Load injection scenarios

In [7]:
import numpy as np
import pandas as pd
import array
import csv
import math
import os.path
import random
import time
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import scipy.io
from pypower.api import ext2int, int2ext, makeYbus
PATH = "/Users/babaktaheri/Desktop/OLDF/Single-phase"


TEST_CASE           =   '906'


def load_mat():
    mat = scipy.io.loadmat(f"{PATH}/data/case{TEST_CASE}.mat")
    bus = mat['ans']['bus'][0][0]
    baseMVA = mat['ans']['baseMVA'][0][0][0][0]
    gen = mat['ans']['gen'][0][0]
    branch = mat['ans']['branch'][0][0]
    keys = ['baseMVA', 'bus', 'gen', 'branch']
    values = [baseMVA, bus, gen, branch]
    ppc = dict(zip(keys, values))
    ppc = ext2int(ppc)
    baseMVA, bus, gen, branch = ppc['baseMVA'], ppc['bus'], ppc['gen'], ppc['branch']
    Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)

    return baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc


def load_gen_data(filename):
    gen_data = pd.read_csv(filename, sep=',')
    return gen_data


def process_bus_and_gen(bus, gen, baseMVA, branch, ppc):
    Just_load = bus[np.logical_or(bus[:, 2] != 0, bus[:, 3] != 0)]
    load_buses=Just_load[:,0]
    load_buses=np.reshape(load_buses,(len(load_buses),1))

    n_buses = len(ppc['bus'])
    n_gens = len(ppc['gen'])
    n_branches = len(ppc['branch'])
    r = branch[:, 2]
    x = branch[:, 3]
    gen_buses = gen[:, 0]
    gen_values = gen[:, 1]
    Q_values = gen[:, 2]


    Pgen = np.zeros(n_buses)
    Qgen = np.zeros(n_buses)

    for i in range(len(gen_buses)):
        bus_idx = int(gen_buses[i])  
        Pgen[bus_idx] += gen_values[i]
        Qgen[bus_idx] += Q_values[i]

    #Depends on ...    
    Pgen  = Pgen / baseMVA
    Qgen  = Qgen / baseMVA

    Pgen = np.reshape(Pgen,(n_buses,1))    
    Qgen = np.reshape(Qgen,(n_buses,1))    

    ref_bus = np.where(bus[:, 1] == 3)[0][0]

    return load_buses, n_buses, n_gens, n_branches, r, x, Pgen,Qgen, ref_bus


baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc = load_mat()
load_buses, n_buses, n_gens, n_branches, r, x, Pgen,Qgen, ref_bus = process_bus_and_gen(bus, gen, baseMVA, branch, ppc)

def compute_and_save_pinj():
    
    baseMVA, bus, gen, branch, _, _, _, ppc = load_mat()
    P_load_all_scenarios = np.load(f"{PATH}/data/High_P_load_scenarios_{TEST_CASE}bus.npy", allow_pickle=True)
    Q_load_all_scenarios = np.load(f"{PATH}/data/High_Q_load_scenarios_{TEST_CASE}bus.npy", allow_pickle=True)

    load_buses, n_buses, _, _, _, _, Pgen,Qgen, ref_bus = process_bus_and_gen(bus, gen, baseMVA, branch, ppc)

    P_inj_all_scenarios = []
    for s in range(P_load_all_scenarios.shape[0]): 
        aaa = P_load_all_scenarios[s, :]

        Demand = np.zeros(n_buses)
        for i in range(len(load_buses)):
            bus_idx = int(load_buses[i][0])  

            Demand[bus_idx] = aaa[i]
        Demand = np.reshape(Demand, (n_buses, 1))
        P_inj = Pgen - Demand
        P_inj = np.delete(P_inj, ref_bus, 0)
        P_inj = np.asmatrix(P_inj)
        P_inj_all_scenarios.append(P_inj)
        
    Q_inj_all_scenarios = []
    for s in range(Q_load_all_scenarios.shape[0]): 
        aaa = Q_load_all_scenarios[s, :]

        Demand = np.zeros(n_buses)
        for i in range(len(load_buses)):
            bus_idx = int(load_buses[i][0])  
            Demand[bus_idx] = aaa[i]
        Demand = np.reshape(Demand, (n_buses, 1))
        Q_inj = Qgen - Demand
        Q_inj = np.delete(Q_inj, ref_bus, 0)
        Q_inj = np.asmatrix(Q_inj)
        Q_inj_all_scenarios.append(Q_inj)        
        
    np.save(f"{PATH}/data/High_P_inj_all_scenarios_{TEST_CASE}bus.npy", P_inj_all_scenarios)
    np.save(f"{PATH}/data/High_Q_inj_all_scenarios_{TEST_CASE}bus.npy", Q_inj_all_scenarios)

compute_and_save_pinj()

# Uniform scenarios

In [8]:
import numpy as np

N = 10000

def generate_scenarios_uniform(P_load, Q_load, N):
    power_factor = P_load / np.sqrt(P_load**2 + Q_load**2)

    P_load_scenarios = np.zeros((N, P_load.shape[0]))
    Q_load_scenarios = np.zeros((N, Q_load.shape[0]))

    for idx in range(P_load.shape[0]):
        if P_load[idx, 0] != 0:
            P_load_scenarios[:, idx] = np.random.uniform(0, 1.5 * P_load[idx, 0], N)
            Q_load_scenarios[:, idx] = P_load_scenarios[:, idx] * np.tan(np.arccos(power_factor[idx, 0]))

    return P_load_scenarios, Q_load_scenarios

P_load_scenarios_uniform, Q_load_scenarios_uniform = generate_scenarios_uniform(P_load, Q_load, N)

np.save(f"{PATH}/data/P_load_all_scenarios_uniform_{TEST_CASE}bus.npy", P_load_scenarios_uniform)
np.save(f"{PATH}/data/Q_load_all_scenarios_uniform_{TEST_CASE}bus.npy", Q_load_scenarios_uniform)


In [9]:
import numpy as np
import pandas as pd
import array
import csv
import math
import os.path
import random
import time
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import scipy.io
from pypower.api import ext2int, int2ext, makeYbus
PATH = "/Users/babaktaheri/Desktop/OLDF/Single-phase"


TEST_CASE           =   '906'


def load_mat():
    mat = scipy.io.loadmat(f"{PATH}/data/case{TEST_CASE}.mat")
    bus = mat['ans']['bus'][0][0]
    baseMVA = mat['ans']['baseMVA'][0][0][0][0]
    gen = mat['ans']['gen'][0][0]
    branch = mat['ans']['branch'][0][0]
    keys = ['baseMVA', 'bus', 'gen', 'branch']
    values = [baseMVA, bus, gen, branch]
    ppc = dict(zip(keys, values))
    ppc = ext2int(ppc)
    baseMVA, bus, gen, branch = ppc['baseMVA'], ppc['bus'], ppc['gen'], ppc['branch']
    Ybus, Yf, Yt = makeYbus(baseMVA, bus, branch)

    return baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc


def load_gen_data(filename):
    gen_data = pd.read_csv(filename, sep=',')
    return gen_data


def process_bus_and_gen(bus, gen, baseMVA, branch, ppc):
    Just_load = bus[np.logical_or(bus[:, 2] != 0, bus[:, 3] != 0)]
    load_buses=Just_load[:,0]
    load_buses=np.reshape(load_buses,(len(load_buses),1))

    n_buses = len(ppc['bus'])
    n_gens = len(ppc['gen'])
    n_branches = len(ppc['branch'])
    r = branch[:, 2]
    x = branch[:, 3]
    gen_buses = gen[:, 0]
    gen_values = gen[:, 1]
    Q_values = gen[:, 2]


    Pgen = np.zeros(n_buses)
    Qgen = np.zeros(n_buses)

    for i in range(len(gen_buses)):
        bus_idx = int(gen_buses[i])  
        Pgen[bus_idx] += gen_values[i]
        Qgen[bus_idx] += Q_values[i]

    #Depends on ...    
    Pgen  = Pgen / baseMVA
    Qgen  = Qgen / baseMVA

    Pgen = np.reshape(Pgen,(n_buses,1))    
    Qgen = np.reshape(Qgen,(n_buses,1))    

    ref_bus = np.where(bus[:, 1] == 3)[0][0]

    return load_buses, n_buses, n_gens, n_branches, r, x, Pgen,Qgen, ref_bus


baseMVA, bus, gen, branch, Ybus, Yf, Yt, ppc = load_mat()
load_buses, n_buses, n_gens, n_branches, r, x, Pgen,Qgen, ref_bus = process_bus_and_gen(bus, gen, baseMVA, branch, ppc)

def compute_and_save_pinj():
    
    baseMVA, bus, gen, branch, _, _, _, ppc = load_mat()
    P_load_all_scenarios = np.load(f"{PATH}/data/P_load_all_scenarios_uniform_{TEST_CASE}bus.npy", allow_pickle=True)
    Q_load_all_scenarios = np.load(f"{PATH}/data/Q_load_all_scenarios_uniform_{TEST_CASE}bus.npy", allow_pickle=True)

    load_buses, n_buses, _, _, _, _, Pgen,Qgen, ref_bus = process_bus_and_gen(bus, gen, baseMVA, branch, ppc)

    P_inj_all_scenarios = []
    for s in range(P_load_all_scenarios.shape[0]): 
        aaa = P_load_all_scenarios[s, :]

        Demand = np.zeros(n_buses)
        for i in range(len(load_buses)):
            bus_idx = int(load_buses[i][0])  

            Demand[bus_idx] = aaa[i]
        Demand = np.reshape(Demand, (n_buses, 1))
        P_inj = Pgen - Demand
        P_inj = np.delete(P_inj, ref_bus, 0)
        P_inj = np.asmatrix(P_inj)
        P_inj_all_scenarios.append(P_inj)
        
    Q_inj_all_scenarios = []
    for s in range(Q_load_all_scenarios.shape[0]): 
        aaa = Q_load_all_scenarios[s, :]

        Demand = np.zeros(n_buses)
        for i in range(len(load_buses)):
            bus_idx = int(load_buses[i][0])  

            Demand[bus_idx] = aaa[i]
        Demand = np.reshape(Demand, (n_buses, 1))
        Q_inj = Qgen - Demand
        Q_inj = np.delete(Q_inj, ref_bus, 0)
        Q_inj = np.asmatrix(Q_inj)
        Q_inj_all_scenarios.append(Q_inj)        
        
    np.save(f"{PATH}/data/P_inj_all_scenarios_uniform_{TEST_CASE}bus.npy", P_inj_all_scenarios)
    np.save(f"{PATH}/data/Q_inj_all_scenarios_uniform_{TEST_CASE}bus.npy", Q_inj_all_scenarios)

compute_and_save_pinj()